In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import normalize
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
%matplotlib inline

In [2]:
#* Read `crypto_data.csv` into Pandas. The dataset was obtained from [CryptoCompare](https://min-api.cryptocompare.com/data/all/coinlist).

file = ('crypto_data.csv')
df = pd.read_csv(file)

In [3]:
df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1252 entries, 0 to 1251
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1252 non-null   object 
 1   CoinName         1252 non-null   object 
 2   Algorithm        1252 non-null   object 
 3   IsTrading        1252 non-null   bool   
 4   ProofType        1252 non-null   object 
 5   TotalCoinsMined  744 non-null    float64
 6   TotalCoinSupply  1252 non-null   object 
dtypes: bool(1), float64(1), object(5)
memory usage: 60.0+ KB


In [5]:
# Discard all cryptocurrencies that are not being traded. In other words, filter for currencies that are currently being traded. Once you have done this, drop the `IsTrading` column from the dataframe.

df = df.drop(df.index[df['IsTrading'] == False]).dropna()
df = df.drop(columns=['IsTrading'])
df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 685 entries, 0 to 1247
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       685 non-null    object 
 1   CoinName         685 non-null    object 
 2   Algorithm        685 non-null    object 
 3   ProofType        685 non-null    object 
 4   TotalCoinsMined  685 non-null    float64
 5   TotalCoinSupply  685 non-null    object 
dtypes: float64(1), object(5)
memory usage: 37.5+ KB


In [7]:
# Remove all rows that have at least one null value.
for column in df.columns:
    print(f"Column {column} has {df[column].isnull().sum()} null values")


Column Unnamed: 0 has 0 null values
Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 0 null values
Column TotalCoinSupply has 0 null values


In [8]:
# Filter for cryptocurrencies that have been mined. That is, the total coins mined should be greater than zero.
df = df.loc[df["TotalCoinsMined"] >= 0]
df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 684 entries, 0 to 1247
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       684 non-null    object 
 1   CoinName         684 non-null    object 
 2   Algorithm        684 non-null    object 
 3   ProofType        684 non-null    object 
 4   TotalCoinsMined  684 non-null    float64
 5   TotalCoinSupply  684 non-null    object 
dtypes: float64(1), object(5)
memory usage: 37.4+ KB


In [10]:
# In order for your dataset to be comprehensible to a machine learning algorithm, its data should be numeric. Since the coin names do not contribute to the analysis of the data, delete the `CoinName` from the original dataframe.

# Create a new DataFrame that holds only the cryptocurrencies names.
coinNames = df['CoinName']
coinNames_df = pd.DataFrame(coinNames)
coinNames_df

,CoinName
0,42 Coin
2,404Coin
4,808
5,EliteCoin
7,Bitcoin
...,...
1238,ZEPHYR
1242,Gapcoin
1245,Beldex
1246,Horizen


In [11]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df.drop(columns = ['CoinName'], inplace=True)
df

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
1238,ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Equihash,PoW,7.296538e+06,21000000


In [12]:
df.set_index('Unnamed: 0')

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
808,SHA-256,PoW/PoS,0.000000e+00,0
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 684 entries, 0 to 1247
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       684 non-null    object 
 1   Algorithm        684 non-null    object 
 2   ProofType        684 non-null    object 
 3   TotalCoinsMined  684 non-null    float64
 4   TotalCoinSupply  684 non-null    object 
dtypes: float64(1), object(4)
memory usage: 32.1+ KB


In [14]:
# Your next step in data preparation is to convert the remaining features with text values, `Algorithm` and `ProofType`, into numerical data. 
# To accomplish this task, use Pandas to create dummy variables. Examine the number of rows and columns of your dataset now. How did they change?

df["TotalCoinSupply"] = df["TotalCoinSupply"].astype(float)
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 684 entries, 0 to 1247
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       684 non-null    object 
 1   Algorithm        684 non-null    object 
 2   ProofType        684 non-null    object 
 3   TotalCoinsMined  684 non-null    float64
 4   TotalCoinSupply  684 non-null    float64
dtypes: float64(2), object(3)
memory usage: 32.1+ KB


In [19]:
df = df.drop(columns=['Unnamed: 0'])

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 684 entries, 0 to 1247
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Algorithm        684 non-null    object 
 1   ProofType        684 non-null    object 
 2   TotalCoinsMined  684 non-null    float64
 3   TotalCoinSupply  684 non-null    float64
dtypes: float64(2), object(2)
memory usage: 42.9+ KB


In [25]:
df =pd.get_dummies(df, columns=["Algorithm", "ProofType"])
print(df.shape)
df.head(5)

KeyError: "None of [Index(['Algorithm', 'ProofType'], dtype='object')] are in the [columns]"

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 684 entries, 0 to 1247
Columns: 105 entries, TotalCoinsMined to ProofType_dPoW/PoW
dtypes: float64(2), uint8(103)
memory usage: 101.0 KB


In [27]:
# Standardize your dataset so that columns that contain larger values do not unduly influence the outcome.

df_scaled = StandardScaler().fit_transform(df)
df_scaled

array([[-0.10312268, -0.03826639, -0.03826394, ..., -0.03826394,
        -0.03826394, -0.03826394],
       [-0.07692584, -0.03826624, -0.03826394, ..., -0.03826394,
        -0.03826394, -0.03826394],
       [-0.10312268, -0.03826639, -0.03826394, ..., -0.03826394,
        -0.03826394, -0.03826394],
       ...,
       [-0.07878691, -0.03826599, -0.03826394, ..., -0.03826394,
        -0.03826394, -0.03826394],
       [-0.10294153, -0.03826638, -0.03826394, ..., -0.03826394,
        -0.03826394, -0.03826394],
       [-0.10311949, -0.03826639, -0.03826394, ..., -0.03826394,
        -0.03826394, -0.03826394]])